In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import cv2
from cycler import cycler
from skimage import io
import pims

In [12]:
plt.rcParams.update({
    # Figure
    "figure.figsize": (10, 6),  # Taille par défaut de la figure (largeur, hauteur en pouces)
    "figure.dpi": 100,  # Résolution en points par pouce
    "figure.facecolor": (0, 0, 0, 1),  # Fond de la figure : noir pur
    "figure.edgecolor": "white",  # Bordure de la figure en blanc
    "figure.titlesize": 20,  # Taille de la police pour le titre principal
    "figure.titleweight": "bold",  # Style de la police pour le titre principal : gras

    # Axes
    "axes.facecolor": (0, 0, 0, 1),  # Fond des axes : noir pur
    "axes.edgecolor": "white",  # Bordure des axes en blanc
    "axes.linewidth": 2,  # Épaisseur des bordures des axes
    "axes.titlesize": 16,  # Taille de la police des titres des axes
    "axes.titleweight": "bold",  # Style de la police pour les titres des axes : gras
    "axes.labelsize": 14,  # Taille de la police des étiquettes des axes
    "axes.labelweight": "medium",  # Style de la police des étiquettes : intermédiaire
    "axes.labelcolor": "white",  # Couleur des étiquettes des axes
    "axes.prop_cycle": cycler(color=["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]),  # Cycle des couleurs pour les lignes
    "axes.grid": True,  # Activer la grille
    "axes.grid.axis": "both",  # Grille pour les deux axes (x et y)
    "axes.grid.which": "major",  # Grille pour les ticks principaux
    "grid.color": "gray",  # Couleur des lignes de la grille
    "grid.linewidth": 0.5,  # Épaisseur des lignes de la grille
    "grid.alpha": 0.6,  # Transparence des lignes de la grille

    # Ticks (Graduations)
    "xtick.color": "white",  # Couleur des ticks sur l'axe x
    "ytick.color": "white",  # Couleur des ticks sur l'axe y
    "xtick.labelsize": 16,  # Taille de la police des ticks sur l'axe x
    "ytick.labelsize": 16,  # Taille de la police des ticks sur l'axe y
    "xtick.direction": "in",  # Ticks pointant vers l'intérieur sur l'axe x
    "ytick.direction": "in",  # Ticks pointant vers l'intérieur sur l'axe y
    "xtick.major.size": 8,  # Longueur des ticks principaux sur l'axe x
    "ytick.major.size": 8,  # Longueur des ticks principaux sur l'axe y
    "xtick.minor.size": 4,  # Longueur des ticks secondaires sur l'axe x
    "ytick.minor.size": 4,  # Longueur des ticks secondaires sur l'axe y
    "xtick.major.width": 1.5,  # Épaisseur des ticks principaux sur l'axe x
    "ytick.major.width": 1.5,  # Épaisseur des ticks principaux sur l'axe y

    # Lignes et marqueurs
    # "lines.linewidth": 2,  # Épaisseur par défaut des lignes
    # "lines.linestyle": "-",  # Style par défaut des lignes continues
    # "lines.color": "#1f77b4",  # Couleur par défaut des lignes
    # "lines.marker": "o",  # Marqueur par défaut : cercle
    # "lines.markersize": 8,  # Taille par défaut des marqueurs
    # "lines.markeredgewidth": 1.5,  # Épaisseur du bord des marqueurs
    # "lines.markerfacecolor": "blue",  # Couleur du remplissage des marqueurs

    # Polices
    "font.size": 12,  # Taille globale de la police
    "font.family": "sans-serif",  # Famille de polices par défaut
    "font.sans-serif": ["Arial", "Helvetica", "DejaVu Sans"],  # Liste des polices sans-serif préférées
    "text.color": "white",  # Couleur du texte

    # Légendes
    "legend.loc": "upper right",  # Emplacement par défaut de la légende
    "legend.fontsize": 12,  # Taille de la police pour la légende
    "legend.frameon": True,  # Activer le cadre autour de la légende
    "legend.framealpha": 0.8,  # Transparence du cadre de la légende
    "legend.edgecolor": "white",  # Couleur de la bordure de la légende
    "legend.facecolor": (0.2, 0.2, 0.2, 0.9),  # Fond de la légende : gris foncé semi-transparent

    # Sauvegarde des graphiques
    "savefig.dpi": 300,  # Résolution par défaut pour les fichiers sauvegardés
    "savefig.format": "png",  # Format par défaut pour les fichiers sauvegardés
    "savefig.facecolor": (0, 0, 0, 1),  # Fond des figures sauvegardées : noir pur
    "savefig.edgecolor": "none",  # Pas de bordure pour les figures sauvegardées
    "savefig.transparent": True,  # Fond transparent pour les fichiers sauvegardés

    # Couleurs et cycles
    "image.cmap": "viridis",  # Palette par défaut pour les images
})


Lecture des fichiers pour trouver les data

In [13]:
def read_and_concatenate_hdf5_by_keyword(folder_path: str, keyword: str) -> pd.DataFrame:
    """
    Lit tous les fichiers HDF5 contenant un mot-clé dans leur nom,
    puis concatène les données en une seule DataFrame.

    Parameters:
    - folder_path (str): Chemin du dossier contenant les fichiers HDF5.
    - keyword (str): Mot-clé à rechercher dans les noms des fichiers HDF5.

    Returns:
    - DATA (pd.DataFrame): DataFrame contenant les données concaténées.
    """
    # Liste des fichiers HDF5 contenant le mot-clé
    hdf5_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
                  if f.endswith(".hdf5") and keyword in f]

    if not hdf5_files:
        raise FileNotFoundError(f"Aucun fichier HDF5 contenant '{keyword}' trouvé dans le dossier {folder_path}.")

    print(f"Fichiers trouvés : {hdf5_files}")

    # Lire et concaténer les fichiers
    data_frames = []
    for file in hdf5_files:
        try:
            print(f"Lecture du fichier : {file}")
            df = pd.read_hdf(file)
            data_frames.append(df)
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier {file} : {e}")

    # Concaténer tous les DataFrames en une seule DataFrame
    if data_frames:
        DATA = pd.concat(data_frames, ignore_index=True)
        print(f"Concaténation terminée. Nombre total de lignes : {len(DATA)}")
        return DATA
    else:
        raise ValueError("Aucune donnée n'a pu être lue à partir des fichiers HDF5.")

In [14]:
# Utilisation
folder_path = "/Users/souchaud/Desktop/Analyses/tables_resultats"
# all_metrics_df = read_and_concatenate_hdf5_by_keyword(folder_path, keyword="all_metrics")
# metrics_df = read_and_concatenate_hdf5_by_keyword(folder_path, keyword="x_metrics")
DATA = read_and_concatenate_hdf5_by_keyword(folder_path, keyword="data")

Fichiers trouvés : ['/Users/souchaud/Desktop/Analyses/tables_resultats/CytoOne_HL5_10x_data.hdf5', '/Users/souchaud/Desktop/Analyses/tables_resultats/CytoOne_SorC_10x_data.hdf5']
Lecture du fichier : /Users/souchaud/Desktop/Analyses/tables_resultats/CytoOne_HL5_10x_data.hdf5
Lecture du fichier : /Users/souchaud/Desktop/Analyses/tables_resultats/CytoOne_SorC_10x_data.hdf5
Concaténation terminée. Nombre total de lignes : 3309312


Création d'un gif (ou mp4) de l'ensemble des cellules d'une manip. 

In [ ]:
def import_img_sequences(path, file_extension='.tif'):
    """
    Load image sequences using the Pims library.

    Parameters:
    - path : str
        Path to the directory containing the image files.
    - file_extension : str, optional
        The file extension of the image files (default is '.tif').

    Returns:
    - frames : pims.FramesSequence
        A pims FramesSequence object containing the loaded image frames.
    """
    images_path = os.path.join(path, '*' + file_extension)
    frames = pims.ImageSequence(images_path)

    print(f"{len(frames)} frames loaded from {path}.")
    return frames

def draw_trajectory_progress(frames, traj, pathway_saving, first_frame=0, last_frame=None, dot_size=0.02, dpi=80):
    """
    Draw full trajectories of particles progressively and save the images.

    Parameters:
    - frames : pims.FramesSequence
        The loaded image frames.
    - traj : pd.DataFrame
        DataFrame containing particle trajectories.
    - pathway_saving : str
        Path to save the resulting images.
    - first_frame : int, optional
        First frame to start drawing (default is 0).
    - last_frame : int, optional
        Last frame to stop drawing (default is the total number of frames).
    - dot_size : float, optional
        Size of the dots to represent particles (default is 0.02).
    - dpi : int, optional
        Resolution of the saved images (default is 80).

    Returns:
    - None
    """
    if last_frame is None:
        last_frame = len(frames) - 1

    os.makedirs(pathway_saving, exist_ok=True)
    colors = {particle: np.random.rand(3) for particle in traj['old_particle'].unique()}

    for num in range(first_frame, last_frame + 1):
        fig, ax = plt.subplots(figsize=(20, 20))
        ax.imshow(frames[num], cmap='gray')
        plt.axis('off')

        # Plot all trajectories up to the current frame
        current_traj = traj[traj['frame'] <= num]
        for particle, color in colors.items():
            particle_traj = current_traj[current_traj['old_particle'] == particle]
            ax.plot(particle_traj['x'], particle_traj['y'], color=color, marker='o', markersize=dot_size * 10, alpha=0.8)

        # Save the current frame
        save_path = os.path.join(pathway_saving, f'frame_{num:04d}.png')
        plt.savefig(save_path, dpi=dpi, bbox_inches='tight')
        plt.close(fig)

def create_video_from_frames(pathway_saving, video_name="trajectory.mp4", fps=10):
    """
    Create a video (MP4) from the saved frames.

    Parameters:
    - pathway_saving : str
        Path where the frames are saved.
    - video_name : str, optional
        Name of the output video file (default is 'trajectory.mp4').
    - fps : int, optional
        Frames per second for the video (default is 10).

    Returns:
    - None
    """
    frame_paths = sorted([os.path.join(pathway_saving, f) for f in os.listdir(pathway_saving) if f.endswith(".png")])
    if not frame_paths:
        raise FileNotFoundError(f"No frames found in {pathway_saving}.")

    # Load the first frame to get dimensions
    first_frame = cv2.imread(frame_paths[0])
    height, width, _ = first_frame.shape

    video_path = os.path.join(pathway_saving, video_name)
    video_writer = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for frame_path in frame_paths:
        frame = cv2.imread(frame_path)
        video_writer.write(frame)

    video_writer.release()
    print(f"Video saved to {video_path}")

def creating_gif(datas, experiment_name, pathway_experiment, pathway_saving, first_frame=0, last_frame=240, dot_size=0.02, dpi=200):
    """
    Generate images and a video showing the trajectories of particles.

    Parameters:
    - datas : pd.DataFrame
        DataFrame containing particle trajectories.
    - experiment_name : str
        Name of the experiment.
    - pathway_experiment : str
        Path to the experiment directory.
    - pathway_saving : str
        Path to save the results.
    - first_frame : int, optional
        First frame to process (default is 0).
    - last_frame : int, optional
        Last frame to process (default is 240).
    - dot_size : float, optional
        Size of the dots to represent particles (default is 0.02).
    - dpi : int, optional
        Resolution of the saved images (default is 200).

    Returns:
    - None
    """
    traj = datas[datas['experiment'] == experiment_name]
    frames_folder = [os.path.join(pathway_experiment, f) for f in os.listdir(pathway_experiment) if experiment_name in f][0]
    frames_folder_mosaic = os.path.join(frames_folder, 'mosaic')
    frames = import_img_sequences(frames_folder_mosaic)

    pathway_saving_gif = os.path.join(pathway_saving, experiment_name)
    draw_trajectory_progress(frames, traj, pathway_saving_gif, first_frame, last_frame, dot_size, dpi)

    # Create MP4 video
    create_video_from_frames(pathway_saving_gif, video_name=f"{experiment_name}_trajectory.mp4")


In [ ]:
creating_gif(
    DATA,
    experiment_name='ASMOT205',
    pathway_experiment="/Users/souchaud/Desktop/A_analyser/CytoOne_SorC_10x_faits/",
    pathway_saving="/Users/souchaud/Desktop/Analyses/gif",
    first_frame=0,
    last_frame=340,
    dot_size=0.01,
    dpi=200
)

In [17]:
def import_img_sequences(path, file_extension='.tif', first_frame=0, last_frame=240):
    """
    Load image sequences using the Pims library.

    Parameters:
    - path : str
        Path to the directory containing the image files.
    - file_extension : str, optional
        The file extension of the image files (default is '.tif').

    Returns:
    - frames : pims.FramesSequence
        A pims FramesSequence object containing the loaded image frames.
    """
    images_path = os.path.join(path, '*' + file_extension)
    frames = pims.ImageSequence(images_path)
    
    # Ajuster la plage des frames si nécessaire
    actual_last_frame = min(last_frame, len(frames) - 1)
    print(f"{len(frames)} frames loaded from {path}. Using frames {first_frame} to {actual_last_frame}.")
    return frames[first_frame:actual_last_frame + 1]


def draw_trajectory_progress(frames, traj, pathway_saving, first_frame=0, last_frame=None, dot_size=0.02, dpi=80):
    """
    Draw full trajectories of particles progressively and save the images.

    Parameters:
    - frames : pims.FramesSequence
        The loaded image frames.
    - traj : pd.DataFrame
        DataFrame containing particle trajectories.
    - pathway_saving : str
        Path to save the resulting images.
    - first_frame : int, optional
        First frame to start drawing (default is 0).
    - last_frame : int, optional
        Last frame to stop drawing (default is the total number of frames).
    - dot_size : float, optional
        Size of the dots to represent particles (default is 0.02).
    - dpi : int, optional
        Resolution of the saved images (default is 80).

    Returns:
    - None
    """
    if last_frame is None:
        last_frame = len(frames) - 1
    else:
        last_frame = min(last_frame, len(frames) - 1)  # Limiter à la longueur réelle

    os.makedirs(pathway_saving, exist_ok=True)
    colors = {particle: np.random.rand(3) for particle in traj['old_particle'].unique()}

    for num in range(first_frame, last_frame + 1):
        fig, ax = plt.subplots(figsize=(20, 20))
        ax.imshow(frames[num], cmap='gray')
        plt.axis('off')

        # Plot all trajectories up to the current frame
        current_traj = traj[traj['frame'] <= num]
        for particle, color in colors.items():
            particle_traj = current_traj[current_traj['old_particle'] == particle]
            ax.plot(particle_traj['x'], particle_traj['y'], color=color, marker='o', markersize=dot_size * 10, alpha=0.8)

        # Save the current frame
        save_path = os.path.join(pathway_saving, f'frame_{num:04d}.png')
        plt.savefig(save_path, dpi=dpi, bbox_inches='tight')
        plt.close(fig)


def creating_gif(datas, experiment_name, pathway_experiment, pathway_saving, first_frame=0, last_frame=240, dot_size=0.02, dpi=200):
    """
    Generate images and a video showing the trajectories of particles.

    Parameters:
    - datas : pd.DataFrame
        DataFrame containing particle trajectories.
    - experiment_name : str
        Name of the experiment.
    - pathway_experiment : str
        Path to the experiment directory.
    - pathway_saving : str
        Path to save the results.
    - first_frame : int, optional
        First frame to process (default is 0).
    - last_frame : int, optional
        Last frame to process (default is 240).
    - dot_size : float, optional
        Size of the dots to represent particles (default is 0.02).
    - dpi : int, optional
        Resolution of the saved images (default is 200).

    Returns:
    - None
    """
    traj = datas[datas['experiment'] == experiment_name]
    frames_folder = [os.path.join(pathway_experiment, f) for f in os.listdir(pathway_experiment) if experiment_name in f][0]
    frames_folder_mosaic = os.path.join(frames_folder, 'mosaic')

    # Importer les images avec ajustement de la plage
    frames = import_img_sequences(frames_folder_mosaic, first_frame=first_frame, last_frame=last_frame)

    pathway_saving_gif = os.path.join(pathway_saving, experiment_name)
    draw_trajectory_progress(frames, traj, pathway_saving_gif, first_frame, last_frame, dot_size, dpi)

    # Create MP4 video
    create_video_from_frames(pathway_saving_gif, video_name=f"{experiment_name}_trajectory.mp4")


# Exemple d'utilisation
creating_gif(
    DATA,
    experiment_name='ASMOT205',
    pathway_experiment="/Users/souchaud/Desktop/A_analyser/CytoOne_SorC_10x_faits/",
    pathway_saving="/Users/souchaud/Desktop/Analyses/gif",
    first_frame=0,
    last_frame=340,
    dot_size=0.01,
    dpi=200
)

700 frames loaded from /Users/souchaud/Desktop/A_analyser/CytoOne_SorC_10x_faits/2024_11_07_ASMOT205_AX3_2024_P2_10x_CytoOne_SorC_0611-10h00-0711-15h15/mosaic. Using frames 0 to 340.


: 

Création d'un gif ou mp4 d'une cellule bien ciblée. Seul le point du tracking à chaque image est tracé, toute la trajectoires n'est pas tracée au fur et à mesure

In [ ]:
def create_trajectory_gif_particle_fixed_frame(data, target_particle, pathway_experiment, pathway_saving,
                                               first_frame=0, last_frame=240, dot_size=2, crop_size=None):
    """
    Create an animated GIF of particle trajectories with a fixed cropping frame.

    Parameters:
    ----------
    data : pd.DataFrame
        DataFrame containing trajectory data, must include columns:
        'frame', 'particle', 'x', 'y'.
    pathway_experiment : str
        Path to the main folder containing experiment folders.
    pathway_saving : str
        Path to save the resulting GIF and intermediary images.
    first_frame : int, optional
        First frame to start plotting.
    last_frame : int, optional
        Last frame to end plotting.
    dot_size : int, optional
        Size of the dots representing the particles.
    crop_size : int, optional
        Size of the crop around the target particle (in pixels). Required.
    target_particle : int, optional
        ID of the particle to center the crop on.

    Returns:
    -------
    None
    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Les données doivent être un DataFrame Pandas.")
    if target_particle is None:
        raise ValueError("Vous devez spécifier une particule cible ('target_particle').")
    if crop_size is None:
        raise ValueError("Vous devez spécifier une taille de recadrage ('crop_size').")
    if 'experiment' not in data.columns or 'particle' not in data.columns:
        raise ValueError("Les colonnes 'experiment' et 'particle' doivent exister dans les données.")

    # Isolation des données pour la particule cible
    particle_data = data[data['particle'] == target_particle]
    if particle_data.empty:
        raise ValueError(f"La particule cible {target_particle} n'existe pas dans les données.")

    # Extraction du nom de l'expérience
    experiment_name = particle_data['experiment'].iloc[0]
    print(f"Nom de l'expérience détecté : {experiment_name}")

    # Rechercher le dossier de l'expérience
    experiment_folder = None
    for folder in os.listdir(pathway_experiment):
        if experiment_name in folder:
            experiment_folder = os.path.join(pathway_experiment, folder, "mosaic")
            break

    if experiment_folder is None or not os.path.exists(experiment_folder):
        raise FileNotFoundError(f"Experiment folder containing '{experiment_name}' not found in {pathway_experiment}")

    # Importer les images du dossier 'mosaic'
    image_files = [os.path.join(experiment_folder, f) for f in sorted(os.listdir(experiment_folder)) if f.endswith((".png", ".jpg", ".tif"))]
    frames = image_files[first_frame:last_frame + 1]

    if len(frames) == 0:
        raise FileNotFoundError(f"No images found in {experiment_folder} for the specified frames.")

    # Créer le dossier de sauvegarde s'il n'existe pas
    pathway_saving_exp = os.path.join(pathway_saving, experiment_name) + str(target_particle)
    if not os.path.exists(pathway_saving_exp):
        os.makedirs(pathway_saving_exp)

    # Définir les coordonnées de recadrage fixes à partir de la position initiale de la particule
    frame_data_init = particle_data.iloc[0]
    x_0, y_0 = int(frame_data_init['x']), int(frame_data_init['y'])

    left = max(0, x_0 - crop_size // 2)
    upper = max(0, y_0 - crop_size // 2)
    right = left + crop_size
    lower = upper + crop_size

    # Assurez-vous que les coordonnées restent dans les limites des images
    for frame_number, name_frame in enumerate(frames):
        with Image.open(name_frame) as img:
            img_width, img_height = img.size
            right = min(right, img_width)
            lower = min(lower, img_height)

            cropped_img = img.crop((left, upper, right, lower))

            # Données pour le tracking
            frame_data = particle_data[particle_data['frame'] == (first_frame + frame_number)]
            if frame_data.empty:
                continue
            x, y = int(frame_data['x'].iloc[0]), int(frame_data['y'].iloc[0])
            rel_x, rel_y = x - left, y - upper

            # Création de l'image avec le tracking
            fig, ax = plt.subplots(figsize=(20, 20))
            ax.imshow(cropped_img, cmap='gray')
            ax.plot(rel_x, rel_y, 'ro', markersize=dot_size)
            ax.axis('off')

            # Sauvegarde de l'image
            cropped_img_path = os.path.join(pathway_saving_exp, f"frame_{frame_number:04d}.png")
            plt.savefig(cropped_img_path, bbox_inches='tight')
            plt.close(fig)

    # Création du GIF
    # gif_path = os.path.join(pathway_saving_exp, f"trajectory_particle_{target_particle}.gif")
    # with imageio.get_writer(gif_path, mode='I', duration=0.1) as writer:
    #     for frame_number in range(len(frames)):
    #         frame_path = os.path.join(pathway_saving_exp, f"frame_{frame_number:04d}.png")
    #         writer.append_data(imageio.imread(frame_path))
    # print(f"GIF sauvegardé à : {gif_path}")

    # Création video MP4
        # Création de la vidéo MP4
    video_path = os.path.join(pathway_saving_exp, f"trajectory_particle_{target_particle}.mp4")
    
    # Charger la première image pour déterminer les dimensions
    first_frame_path = os.path.join(pathway_saving_exp, "frame_0000.png")
    if not os.path.exists(first_frame_path):
        raise FileNotFoundError("Le premier cadre n'a pas été généré.")
    
    first_frame = imageio.imread(first_frame_path)
    height, width, layers = first_frame.shape
    
    # Initialisation du writer vidéo
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec pour MP4
    video_writer = cv2.VideoWriter(video_path, fourcc, 10, (width, height))  # 10 fps
    
    # Ajouter chaque image à la vidéo
    for frame_number in range(len(frames)):
        frame_path = os.path.join(pathway_saving_exp, f"frame_{frame_number:04d}.png")
        if os.path.exists(frame_path):
            frame = cv2.imread(frame_path)
            video_writer.write(frame)
    
    # Libération du writer
    video_writer.release()
    print(f"Vidéo MP4 sauvegardée à : {video_path}")

In [ ]:
create_trajectory_gif_particle_fixed_frame(
    data=DATA,
    target_particle=2,
    pathway_experiment="/Users/souchaud/Desktop/A_analyser/CytoOne_SorC_10x_faits/",
    pathway_saving="/Users/souchaud/Desktop/Analyses/gif",
    first_frame=0,
    last_frame=185,
    dot_size=25,
    crop_size=200  # Taille du cadre fixe
)